In [1]:
RDATE = 202112
PDATE = 202111

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

AITAB = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\04_Dataset\\"+str(RDATE)+"\HP_"+str(RDATE)+".txt", sep = "\t", header = 0, error_bad_lines = False, encoding='cp1252')
AITAB_P = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\04_Dataset\\"+str(PDATE)+"\HP_"+str(PDATE)+".txt", sep = "\t", header = 0, error_bad_lines = False, encoding='cp1252')

AUTO = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\04_Dataset\\"+str(RDATE)+"\Auto_NCIF.xlsx", sheet_name='Sheet1')

def RATING(x):
    if x=='R1':
        return '01. R1'
    elif x=='R2':
        return '02. R2'
    elif x=='R3':
        return '03. R3'
    elif x=='R4':
        return '04. R4'
    elif x=='R5':
        return '05. R5'
    elif x=='R6':
        return '06. R6'
    elif x=='R7':
        return '07. R7'
    elif x=='R8':
        return '08. R8'
    elif x=='R9':
        return '09. R9'
    elif x=='R10':
        return '10. R10'
    elif x=='R11':
        return '11. R11'
    else: 
        return '13. Unrated'
    
def INCOM(x):
    if x=='Up to RM999':
        return '01. Up to RM3,000'
    elif x=='RM1,000 to RM1,999':
        return '01. Up to RM3,000'
    elif x=='RM2,000 to RM2,999':
        return '01. Up to RM3,000'
    elif x=='RM3,000 to RM3,999':
        return '02. Above RM3,000 to RM5,000'
    elif x=='RM4,000 to RM4,999':
        return '02. Above RM3,000 to RM5,000'
    elif x=='RM5,000 to RM5,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM6,000 to RM6,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM7,000 to RM7,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM8,000 to RM8,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM9,000 to RM9,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM10,000 to RM10,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM11,000 to RM14,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM15,000 to RM20,000':
        return '05. Above RM15,000 to RM20,000'
    elif x=='Above RM 20,000':
        return '06. Above RM20,000'
    else: 
        return 'Not Populated'

AUTO.columns = AUTO.columns.str.replace(" ", "_")
AUTO.rename(columns = {'Ac_No':'Account_No'}, inplace = True)

AUTO.Account_No = AUTO.Account_No.astype(str)
AUTO.Account_No = AUTO.Account_No.str.upper()
AUTO.Account_No = AUTO.Account_No.str.strip()

AITAB.sort_values(by=["Account_No"],ascending=True, inplace=True)
AITAB.Account_No = AITAB.Account_No.astype(str)
AITAB.Account_No = AITAB.Account_No.str.upper()
AITAB.Account_No = AITAB.Account_No.str.strip()

AITAB_P.Account_No = AITAB_P.Account_No.astype(str)
AITAB_P.Account_No = AITAB_P.Account_No.str.upper()
AITAB_P.Account_No = AITAB_P.Account_No.str.strip()

AITAB1 = AITAB.merge(AUTO, on="Account_No", how = 'left', indicator = True)

def FOUND(_merge):
    if _merge == 'both':
        return 'Y'
    else:
        return 'N'
AITAB1['FOUND'] = AITAB1._merge.apply(FOUND)

#-----------------------------------------AT001------------------------------

AT001 = AITAB1.iloc[np.where(AITAB1.FACILITY_TYPE=='I')]
AT001 = AT001.drop(columns=["_merge"])
AT001.rename(columns = {'RISK_GRADE_VAL':'RISK_GRADE_VAL1','Income_Grp_Desc':'Income_Grp_Desc1','Occ_Desc':'Occ_Desc1'}, inplace = True)

AT001['RISK_GRADE_VAL'] = AT001['RISK_GRADE_VAL1'].apply(RATING)

AT001.NPL_Flag = AT001.NPL_Flag.str.strip()

AT001.loc[AT001.NPL_Flag == 'Y','RISK_GRADE_VAL'] = '12. IF'
AT001['OSM'] = AT001.BNM_BALANCE/1000000
AT001['NOA'] = 1


AT001.Month_In_Arrears = AT001.Month_In_Arrears.str.strip()

AT001.loc[(AT001.NPL_Flag != 'Y')&(AT001.Month_In_Arrears == '2'),'OSM_MIA2'] = AT001.BNM_BALANCE/1000000
AT001.loc[(AT001.NPL_Flag != 'Y')&(AT001.Month_In_Arrears == '2'),'NOA_MIA2'] = 1

AT001.loc[(AT001.NPL_Flag != 'Y')&(AT001.Month_In_Arrears == '3'),'OSM_MIA3'] = AT001.BNM_BALANCE/1000000
AT001.loc[(AT001.NPL_Flag != 'Y')&(AT001.Month_In_Arrears == '3'),'NOA_MIA3'] = 1

AT001.loc[AT001.NPL_Flag == 'Y','OSM_IF'] = AT001.BNM_BALANCE/1000000
AT001.loc[AT001.NPL_Flag == 'Y','NOA_IF'] = 1

AT001.loc[(AT001.FOUND == 'Y')&(AT001.NNPL_Flg == 'Y'),'OSM_NCIF'] = AT001.BNM_BALANCE/1000000
AT001.loc[(AT001.FOUND == 'Y')&(AT001.NNPL_Flg == 'Y'),'NOA_NCIF'] = 1

AT001['Income_Grp_Desc'] = AT001['Income_Grp_Desc1'].apply(INCOM)

AT001.Occ_Desc1 = AT001.Occ_Desc1.str.strip()

def Occ_Desc(x):
    if x == 'OTHERS':
        return 'ZA_OTHERS'
    elif x == '?':
        return 'ZB_NOT POPULATES'
    else:
        return x
AT001['Occ_Desc'] = AT001.Occ_Desc1.apply(Occ_Desc)

#AT001.loc[AT001.Occ_Desc1 == 'OTHERS','Occ_Desc'] = 'ZA_OTHERS'
#AT001.loc[AT001.Occ_Desc1 == '?','Occ_Desc'] = 'ZB_NOT POPULATED'
#AT001.loc[(AT001.Occ_Desc1 != 'OTHERS')|(AT001.Occ_Desc1 != '?'),'Occ_Desc'] = AT001.Occ_Desc1

AT001.Gender_Desc = AT001.Gender_Desc.str.strip()

def GENDER(Gender_Desc):
    if Gender_Desc =='FEMALE':
        return '01. Female'
    elif Gender_Desc =='MALE':
        return '02. Male'
    else:
        return '04. Not Populated'
AT001['GENDER'] = AT001.Gender_Desc.apply(GENDER)

def MOF(APPROVED_LIMIT,CASH_PRICE):
    if APPROVED_LIMIT == 0:
        return 0
    elif CASH_PRICE == 0:
        return 0
    else:
        return APPROVED_LIMIT/CASH_PRICE
AT001['MOF'] = AT001.apply(lambda x: MOF(x['APPROVED_LIMIT'], x['CASH_PRICE']), axis=1)

#============================================================================

def MOF_BAND(MOF):
    if MOF < 0.7: 
        return "01.<70%"
    elif MOF >= 0.7 and MOF < 0.8: 
        return "02.70% - <80%"
    elif MOF >= 0.8 and MOF < 0.85: 
        return "03.80% - <85%"
    elif MOF >= 0.85:
        return "04.>=85%"
    else:
        return "05.NA"
AT001['MOF_BAND'] = AT001['MOF'].apply(MOF_BAND)

AT001['TENURE_YEARS'] = AT001.Loan_Tenure_In_Mths/12

def Tenure_Band(x): 
    if x < 4: 
        return '01.<=3 YEARS'
    elif x >= 4 and x < 6: 
        return '02.4 TO 5 YEARS'
    elif x >= 6 and x < 8: 
        return '03.6 TO 7 YEARS'
    elif x >= 8:
        return '04.>=8 YEARS'
AT001['Tenure_Band'] = AT001['TENURE_YEARS'].apply(Tenure_Band)

AT001.VEHICLE_MAKE_DESC = AT001.VEHICLE_MAKE_DESC.str.strip()
AT001.VEHICLE_MAKE_DESC = AT001.VEHICLE_MAKE_DESC.str.upper()

AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["PERODUA"]),'VEHICLE_GROUP'] = "01.PERODUA"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["TOYOTA","TOYOTA MACHINERY"]),'VEHICLE_GROUP'] = "02.TOYATA"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["PROTON"]),'VEHICLE_GROUP'] = "03.PROTON"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["HONDA","HONDA M/CYCLE"]),'VEHICLE_GROUP'] = "04.HONDA"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["NISSAN"]),'VEHICLE_GROUP'] = "05.NISSAN"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["M/BENZ"]),'VEHICLE_GROUP'] = "06.MERCEDES BENZ"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["BMW","BMW M/CYCLE"]),'VEHICLE_GROUP'] = "07.BMW"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["NAZA","NAZA M/CYCLE"]),'VEHICLE_GROUP'] = "08.NAZA"
AT001.loc[AT001.VEHICLE_MAKE_DESC.isin(["MITSUBISHI"]),'VEHICLE_GROUP'] = "09.MITSUBISHI"
AT001.VEHICLE_GROUP.fillna(-9999, inplace=True)
AT001.loc[AT001.VEHICLE_GROUP.isin([-9999]),'VEHICLE_GROUP'] = "10.OTHERS"

#	/*20160120 - BNM requested to segregate out the recon tagging*/

AT001.GOOD_STATUS = AT001.GOOD_STATUS.str.strip()

def VEHICLE_STATUS(x):
    if x == "N":
        return "01.NEW"
    elif x == "U":
        return "02.USED"
    elif x == "R": 
        return "03.RECOND"
    else:
        return "04.NOT POPULATED"
AT001['VEHICLE_STATUS'] = AT001['GOOD_STATUS'].apply(VEHICLE_STATUS)

AT001.VEHICLE_STATUS = AT001.VEHICLE_STATUS.str.strip()

AT001.loc[(AT001.VEHICLE_STATUS.isin(["01.NEW"]))&(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "01.NEW-NATIONAL"
AT001.loc[(AT001.VEHICLE_STATUS.isin(["01.NEW"]))&~(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "02.NEW-NON NATIONAL"
AT001.loc[(AT001.VEHICLE_STATUS.isin(["02.USED"]))&(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "03.USED-NATIONAL"
AT001.loc[(AT001.VEHICLE_STATUS.isin(["02.USED"]))&~(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "04.USED-NON NATIONAL"
AT001.loc[(AT001.VEHICLE_STATUS.isin(["03.RECOND"]))&(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "05.RECOND-NATIONAL"
AT001.loc[(AT001.VEHICLE_STATUS.isin(["03.RECOND"]))&~(AT001.VEHICLE_MAKE_DESC.isin(["PERODUA","PROTON"])),'VEHICLE_DETAIL'] = "06.RECOND-NON NATIONAL"
AT001.VEHICLE_DETAIL.fillna(-9999, inplace=True)
AT001.loc[AT001.VEHICLE_DETAIL.isin([-9999]),'VEHICLE_DETAIL'] = "07.NOT POPULATED"

AT001.CUSTOMER_SEGMENT = AT001.CUSTOMER_SEGMENT.str.strip()

def CUSTOMER_SEG(CUSTOMER_SEGMENT):
    if CUSTOMER_SEGMENT=='HIGH NETWORTH':
        return '01. HNW'
    elif CUSTOMER_SEGMENT=='AFFLUENT':
        return '02. Affluent'
    elif CUSTOMER_SEGMENT=='EMERGING AFFLUENT':
        return '03. Emerging Affluent'
    elif CUSTOMER_SEGMENT=='MASS':
        return '04. Mass'
    elif CUSTOMER_SEGMENT=='PREMIER':
        return '05. Premier'
    elif CUSTOMER_SEGMENT=='PREFERRED':
        return '06. Preferred'
    elif CUSTOMER_SEGMENT=='CLASSIC':
        return '07. Classic'
    else:
        return '08. Not Populated'
AT001['CUSTOMER_SEG'] = AT001.CUSTOMER_SEGMENT.apply(CUSTOMER_SEG)

AT001['DSR_RATIO'] = AT001['DSR_RATIO'].str.strip()

AT001.loc[AT001.DSR_RATIO.isin(['?']),'DSR_RATIO'] = -9999

AT001['DSR_RATIO'] = AT001['DSR_RATIO'].map(int)

def DSR_BAND(x):
    if x <= 0:
        return "06.NA"
    elif x > 0 and x <50:
        return "01.<50%"
    elif x >= 50 and x < 70:
        return "02.50-<70%"
    elif x >= 70 and x <90:
        return "03.70-<90%"
    elif x >= 90 and x <100:
        return "04.90-<100%"
    elif x >=100:
        return "05.>=100%"
    else:
        return "06.NA"
AT001['DSR_BAND'] = AT001.DSR_RATIO.apply(DSR_BAND)

PMNTH_MIA = AITAB_P.iloc[np.where(AITAB_P.FACILITY_TYPE=='I')][['Account_No','Month_In_Arrears','BNM_BALANCE']].rename(columns={'Month_In_Arrears': 'Month_In_Arrears_P','BNM_BALANCE':'BNM_BALANCE_P'})

PMNTH_MIA1 = PMNTH_MIA.sort_values(by=['Account_No'],ascending=True).drop_duplicates() #subset=['Account_No']

AT002 = AT001.sort_values(by=['Account_No'],ascending=True).drop_duplicates() #subset=['Account_No']

AT003 = AT002.merge(PMNTH_MIA1, on = 'Account_No', how = 'left' , indicator = True)

AT003['Month_In_Arrears'] = AT003['Month_In_Arrears'].str.strip()
AT003['Month_In_Arrears_P'] = AT003['Month_In_Arrears_P'].str.strip()

AT003.loc[AT003.Month_In_Arrears.isin(['?']),'Month_In_Arrears'] = -9999
AT003.loc[AT003.Month_In_Arrears_P.isin(['?']),'Month_In_Arrears_P'] = -9999

AT003.Month_In_Arrears.fillna(-9999, inplace=True)
AT003.Month_In_Arrears_P.fillna(-9999, inplace=True)

AT003['Month_In_Arrears'] = AT003['Month_In_Arrears'].map(int)
AT003['Month_In_Arrears_P'] = AT003['Month_In_Arrears_P'].map(int)

def WIFLG(_merge,Month_In_Arrears,Month_In_Arrears_P):
    if _merge == 'both':
        if Month_In_Arrears > Month_In_Arrears_P:
            return 'Forward Flow'
        elif Month_In_Arrears == 0 and Month_In_Arrears_P > Month_In_Arrears:
            return 'Regular'
        elif Month_In_Arrears > 0 and Month_In_Arrears_P > Month_In_Arrears:
            return 'Push Back'
        elif Month_In_Arrears == Month_In_Arrears_P:
            return 'Standardized'
    if _merge == 'left_only':
        return 'New Account'
AT003['WIFLG'] = AT003.apply(lambda x: WIFLG(x['_merge'],x['Month_In_Arrears'],x['Month_In_Arrears_P']), axis=1)


AT003_1 = AT003.drop(columns=["_merge"])

AT004 = AT002.merge(PMNTH_MIA1, on = 'Account_No', how = 'right')

AT004['Month_In_Arrears'] = AT004['Month_In_Arrears'].str.strip()
AT004['Month_In_Arrears_P'] = AT004['Month_In_Arrears_P'].str.strip()

AT004.loc[AT004.Month_In_Arrears.isin(['?']),'Month_In_Arrears'] = -9999
AT004.loc[AT004.Month_In_Arrears_P.isin(['?']),'Month_In_Arrears_P'] = -9999

AT004.Month_In_Arrears.fillna(-9999, inplace=True)
AT004.Month_In_Arrears_P.fillna(-9999, inplace=True)

AT004['Month_In_Arrears'] = AT004['Month_In_Arrears'].map(int)
AT004['Month_In_Arrears_P'] = AT004['Month_In_Arrears_P'].map(int)

AT004.loc[(AT004.Month_In_Arrears == 2) & (AT004.Month_In_Arrears_P == 1), 'OSM_MIA2_Migration'] = AT004.BNM_BALANCE/1000000
AT004.loc[(AT004.Month_In_Arrears == 3) & (AT004.Month_In_Arrears_P == 2), 'OSM_MIA3_Migration'] = AT004.BNM_BALANCE/1000000
AT004.loc[(AT004.Month_In_Arrears == 4) & (AT004.Month_In_Arrears_P == 3), 'OSM_MIA4_Migration'] = AT004.BNM_BALANCE/1000000
AT004['OSM'] = AT004.BNM_BALANCE/1000000
AT004['NOA'] = 1



D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
AT003_1.head(1)

,As_Of_Date,Account_No,Best_EL_Estimate,EAD,RWA,EL,FACILITY_TYPE,SOURCE_PROD_TYPE_CD,Cus_No,Age,Age_Group,APPLICANT_TYPE,NAME,Gender_Desc,Income_Grp_Desc1,HPS_TOTAL_MTHLY_INCOME,Occ_Desc1,Occ_Sector_Desc,NPL_Flag,APPROVED_LIMIT,BNM_BALANCE,BRANCH_CODE,BRANCH_DESC,DATE_AA_APPROVED,ISSUE_DATE,Loan_Tenure_In_Mths,HPS_Approved_Loan_Tenure_Mths,CASH_PRICE,Month_On_Book,Month_On_Book_Loan_Range,Application_Score_Risk_Grade,MTHS_INSTALLMENT_IN_ARREARS,MODEL_DESC,NATURE_OF_BUSINESS,GOOD_STATUS,VEHICLE_MAKE_CD,VEHICLE_MAKE_DESC,NEW_ASSET_FINANCED_FLG,MBB_Economic_Sector_Desc,CUSTOMER_SEGMENT,NPL_DATE,RESCHEDULE_FLG,RESTRUCTURE_FLG,BRANCH_ZONE,STATE,CUR_GROSS_RATE,APPLICATION_SCORE,Month_In_Arrears,RISK_GRADE_VAL1,DSR_RATIO,MRGN_OF_ADVN,SCORE_VAL,SCORE_RATG,SCORE_DT,EMP_TYPE_CD,EMP_TYPE_DESC,CORONA_FLAG,CORONA_TAG_DATE,INDUL_STRT_DATE,INDUL_END_DATE,CORONA2_FLAG,CORONA2_TAG_DATE,Installment_Amt,NEW_IC_NUM,OTHER_ID,OTHER_ID_TYPE,STAGE_CLASSIFICATION,ECL_CLOSING_FY_MYR,EAD_MYR,Borrower,CD1_BOS,NCIL_MIA,NCIL_Status,NNPL_Flg,ageing_range,FOUND,RISK_GRADE_VAL,OSM,NOA,OSM_MIA2,NOA_MIA2,OSM_MIA3,NOA_MIA3,OSM_IF,NOA_IF,OSM_NCIF,NOA_NCIF,Income_Grp_Desc,Occ_Desc,GENDER,MOF,MOF_BAND,TENURE_YEARS,Tenure_Band,VEHICLE_GROUP,VEHICLE_STATUS,VEHICLE_DETAIL,CUSTOMER_SEG,DSR_BAND,Month_In_Arrears_P,BNM_BALANCE_P,WIFLG
0,31/12/2021,789707002671,0.00000000,1166.63000000,238.12000000,1.60000000,I,N2,5497221,48,45 to <50 years,?,NORAZIAN BINTI SAMSUDDIN,FEMALE,"RM1,000 to RM1,999",?,CLEANERS AND HELPERS,ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES,N,11127.07,1166.63,9707,AFC BANDAR BARU UDA,03/11/2008,10/05/2003,61,60,32000.0,224,Above 72 months,?,?,SAGA ISWARA 1.3S,?,U,21,PROTON,N,?,MASS,?,Y,N,JOHOR/MELAKA,JOHOR,6.3,?,-9999,R2,-9999,?,?,?,?,113,Private Employee,?,?,?,?,?,?,244.0,730926016482,A2573011,IC,STAGE1,0.69,1166.63,NaN,NaN,NaN,NaN,NaN,NaN,N,02. R2,1.17e-03,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01. Up to RM3,000",CLEANERS AND HELPERS,01. Female,0.35,01.<70%,5.08,02.4 TO 5 YEARS,03.PROTON,02.USED,03.USED-NATIONAL,04. Mass,06.NA,-9999,1166.63,Standardized


In [19]:
#AT003_1.OSM = AT003_1.OSM.str.strip()
AT003_1.loc[AT003_1.OSM=='?','OSM'] = 0
AT003_1.OSM = AT003_1.OSM.astype(float)

#AT003_1.NOA = AT003_1.NOA.str.strip()
AT003_1.loc[AT003_1.NOA=='?','NOA'] = 0
AT003_1.NOA = AT003_1.NOA.astype(float)

#AT003_1.EXPECTED_LOSS = AT003_1.EXPECTED_LOSS.str.strip()
AT003_1.loc[AT003_1.EL=='?','EL'] = 0
AT003_1.EL = AT003_1.EL.astype(float)

#AT003_1.EAD = AT003_1.EAD.str.strip()
AT003_1.loc[AT003_1.EAD=='?','EAD'] = 0
AT003_1.EAD = AT003_1.EAD.astype(float)

#AT003_1.RWA = AT003_1.RWA.str.strip()
AT003_1.loc[AT003_1.RWA=='?','RWA'] = 0
AT003_1.RWA = AT003_1.RWA.astype(float)


RISK = AT003_1.fillna(0).groupby(['RISK_GRADE_VAL'])[['OSM','NOA','EL','EAD','RWA']].sum().reset_index()


In [25]:
#-------------------------------------------SORTBY(EXCEL)

writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\07_Yearly_Reporting\Budget\BudgetFY21\working - SH\2. AITAB_CFS'+str(RDATE)+'(python).xlsx',engine='xlsxwriter')
    

#AT003_1.to_excel(writer2, sheet_name='Base', index = False)
RISK.to_excel(writer2, sheet_name='Auto', index = False)


writer2.save()

In [20]:
RISK

,RISK_GRADE_VAL,OSM,NOA,EL,EAD,RWA
0,01. R1,34079.01,656005.0,2.74e+07,3.41e+10,5.72e+09
1,02. R2,1527.61,84066.0,3.61e+06,1.53e+09,5.38e+08
2,03. R3,1019.74,17290.0,3.95e+06,1.02e+09,4.46e+08
3,04. R4,2064.58,45385.0,1.15e+07,2.06e+09,1.05e+09
4,05. R5,273.62,12735.0,3.40e+06,2.74e+08,1.90e+08
5,06. R6,480.63,9366.0,9.53e+06,4.81e+08,3.23e+08
6,07. R7,537.51,12732.0,1.77e+07,5.38e+08,3.93e+08
7,08. R8,1302.26,34634.0,8.04e+07,1.30e+09,1.18e+09
8,09. R9,69.61,1696.0,8.30e+06,6.96e+07,8.06e+07
9,10. R10,505.92,14766.0,1.17e+08,5.06e+08,6.38e+08


In [ ]:
#row = ['OSM','NOA','OSM_MIA2','NOA_MIA2','OSM_MIA3','NOA_MIA3','OSM_IF','NOA_IF','OSM_NCIF','NOA_NCIF']

#RISK_GRADE_VAL = AT003_1.groupby(['RISK_GRADE_VAL'])[row].sum().reset_index()
#RISK_GRADE_VAL1 = AT003_1.groupby(['Income_Grp_Desc'])[row].sum().reset_index()
#RISK_GRADE_VAL2 = AT003_1.groupby(['Occ_Desc'])[row].sum().reset_index()
#RISK_GRADE_VAL3 = AT003_1.groupby(['GENDER'])[row].sum().reset_index()
#RISK_GRADE_VAL4 = AT003_1.groupby(['DSR_BAND'])[row].sum().reset_index()
#RISK_GRADE_VAL5 = AT003_1.groupby(['MOF_BAND'])[row].sum().reset_index()
#RISK_GRADE_VAL6 = AT003_1.groupby(['Tenure_Band'])[row].sum().reset_index()
#RISK_GRADE_VAL7 = AT003_1.groupby(['STATE'])[row].sum().reset_index()
#RISK_GRADE_VAL8 = AT003_1.groupby(['VEHICLE_GROUP'])[row].sum().reset_index()
#RISK_GRADE_VAL9 = AT003_1.groupby(['VEHICLE_DETAIL'])[row].sum().reset_index()
#RISK_GRADE_VAL10 = AT003_1.groupby(['VEHICLE_STATUS'])[row].sum().reset_index()
#RISK_GRADE_VAL11 = AT003_1.groupby(['CUSTOMER_SEG'])[row].sum().reset_index()
#RISK_GRADE_VAL12 = AT003_1.groupby(['WIFLG'])[row].sum().reset_index()

#MIGRATION_RATING = AT004.groupby(['Month_In_Arrears_P'])[['OSM','NOA','OSM_MIA2_Migration','OSM_MIA3_Migration','OSM_MIA4_Migration']].sum().reset_index()

#writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\04_AITAB_Financing\04_Dataset\\'+str(RDATE)+'\AITAB_'+str(RDATE)+'(python).xlsx',engine='xlsxwriter')

#RISK_GRADE_VAL.to_excel(writer2, sheet_name='RISK_GRADE_VAL', index = False)
#RISK_GRADE_VAL1.to_excel(writer2, sheet_name='Income_Grp_Desc', index = False)
#RISK_GRADE_VAL2.to_excel(writer2, sheet_name='Occ_Desc', index = False)
#RISK_GRADE_VAL3.to_excel(writer2, sheet_name='GENDER', index = False)
#RISK_GRADE_VAL4.to_excel(writer2, sheet_name='DSR_BAND', index = False)
#RISK_GRADE_VAL5.to_excel(writer2, sheet_name='MOF_BAND', index = False)
#RISK_GRADE_VAL6.to_excel(writer2, sheet_name='TENURE_BAND', index = False)
#RISK_GRADE_VAL7.to_excel(writer2, sheet_name='STATE', index = False)
#RISK_GRADE_VAL8.to_excel(writer2, sheet_name='VEHICLE_GROUP', index = False)
#RISK_GRADE_VAL9.to_excel(writer2, sheet_name='VEHICLE_DETAIL', index = False)
#RISK_GRADE_VAL10.to_excel(writer2, sheet_name='VEHICLE_STATUS', index = False)
#RISK_GRADE_VAL11.to_excel(writer2, sheet_name='CUSTOMER_SEG', index = False)
#RISK_GRADE_VAL12.to_excel(writer2, sheet_name='WIFLG', index = False)

#MIGRATION_RATING.to_excel(writer2, sheet_name='RATING_MIGRATION', index = False)

#writer2.save()